In [1]:
#Importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import glob

%matplotlib inline

In [2]:
def list_images(images, cols = 2, rows = 6, cmap=None):
    """
    Display a list of images in a single figure with matplotlib.
        Parameters:
            images: List of np.arrays compatible with plt.imshow.
            cols (Default = 2): Number of columns in the figure.
            rows (Default = 5): Number of rows in the figure.
            cmap (Default = None): Used to display gray images.
    """
    plt.figure(figsize=(10, 11))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        #Using gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape) == 2 else cmap
        img = cv2.resize(image, (480,270))
        plt.imshow(img, cmap = cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [3]:
def display_image(test_image):
    plt.figure(figsize=(10,11))
    cmap = None
    cmap = 'gray' if len(test_image.shape) == 2 else cmap
    img = cv2.resize(test_image, (480, 270))
    plt.imshow(img, cmap = cmap)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [4]:
#Reading in the test images
test_image = plt.imread('Road-detection/test_images/frame2519_leftImg8bit.jpg')
display_image(test_image)
#list_images(test_images)

FileNotFoundError: [Errno 2] No such file or directory: 'Road-detection/test_images/frame2519_leftImg8bit.jpg'

In [ ]:
def RGB_colour_selection(image):
    """
    Apply color selection to RGB images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #White color mask
    lower_threshold = np.uint8([200, 200, 200])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([130, 130, 0])
    upper_threshold = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [ ]:
rgb_selection_img = RGB_colour_selection(test_image)
display_image(rgb_selection_img)

In [ ]:
def convert_hsl(image):
    """
    Convert RGB images to HSL.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

display_image(convert_hsl(rgb_selection_img))

In [ ]:
def HSL_colour_selection(image):
    """
    Apply color selection to the HSL images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #Convert the input image to HSL
    converted_image = convert_hsl(image)
    
    #White color mask
    lower_threshold = np.uint8([0, 170, 0])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([10, 0, 100])
    upper_threshold = np.uint8([40, 255, 255])
    yellow_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [ ]:
display_image(HSL_colour_selection(rgb_selection_img))

In [ ]:
color_selected_image = RGB_colour_selection(test_image)
display_image(RGB_colour_selection(test_image))

# ***Canny Edge Detection:***
   The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a wide range of edges in images

In [ ]:
#Gray scaling the images
def gray_scale(image):
    """
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
gray_image = gray_scale(color_selected_image)
display_image(gray_image)

In [ ]:
#Applying gaussian smoothing
def gaussian_smoothing(image, kernel_size = 11):
    """
    Apply Gaussian filter to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            kernel_size (Default = 13): The size of the Gaussian kernel will affect the performance of the detector.
            It must be an odd number (3, 5, 7, ...).
    """
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

In [ ]:
blur_image = gaussian_smoothing(gray_image)
display_image(blur_image)

In [ ]:
#Applying canny edge detection
def canny_detector(image, low_threshold = 50, high_threshold = 170):
    """
    Apply Canny Edge Detection algorithm to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            low_threshold (Default = 50).
            high_threshold (Default = 150).
    """
    return cv2.Canny(image, low_threshold, high_threshold)

In [ ]:
edge_detected_image = canny_detector(blur_image)
display_image(edge_detected_image)

In [ ]:
def region_selection(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    mask = np.zeros_like(image)   
    #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #We could have used fixed numbers as the vertices of the polygon,
    #but they will not be applicable to images with different dimesnions.
    rows, cols = image.shape[:2]
    bottom_left  = [cols * 0.1, rows * 0.95]
    top_left     = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.8]
    top_right    = [cols * 0.6, rows * 0.5]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [ ]:
masked_image = region_selection(edge_detected_image)
display_image(masked_image)